In [1]:
import gym
from scipy.special import expit as sigmoid
import numpy as np
import numpy.random as npr
import time
from itertools import count
from collections import deque
import matplotlib.pyplot as plt
from a2c_ppo_acktr import utils

from layer import Layer

In [2]:
class args(object):
    num_hidden = 8
    bandwidth = 3
    eval_interval = None
    log_interval = 10
    use_gae = False
    num_updates = 1e5
    num_steps = 32
    memory_capacity = 32
    clip_param = 0.2
    ppo_epoch = 4
    num_mini_batch = 32
    value_loss_coef = 0.5
    entropy_coef = 0.01
    lr = 1e-8
    eps = 1e-5
    max_grad_norm = 0.2
    gamma = 0.99
    gae_lambda = 0.95
    use_proper_time_limits = False

In [3]:
env = gym.make('CartPole-v0')
input = Layer(4, 1, args.bandwidth, post_width=args.num_hidden, args=args, post_process=None)
hidden = Layer(args.num_hidden, args.bandwidth, args.bandwidth, post_width=1, args=args, post_process=None)
output = Layer(1, args.bandwidth, 1, post_process=sigmoid, args=args)

/usr/local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
episode_rewards = deque(maxlen=10)
done = True

for j in count(1):
    input.clear_memory()
    hidden.clear_memory()
    output.clear_memory()
    if done:
        state = env.reset()
        episode_rewards.append(0)
    for unit in input.units + hidden.units + output.units:
        utils.update_linear_schedule(unit.agent.optimizer, j, args.num_updates, args.lr)
    for step in range(args.num_steps):
        action = output(hidden(input([state[i:i+1] for i in range(4)])))[0][0]
        action = int(action > 0.5)
        state, reward, done, info = env.step(action)
        input.reward(reward)
        hidden.reward(reward)
        output.reward(reward)
        episode_rewards[-1] += reward
        if done:
            input.done()
            hidden.done()
            output.done()
    input.update()
    hidden.update()
    output.update()
    print('Iter: %d, Avg/Max/Min. reward: %0.1f/%0.1f/%0.1f' % (j, sum(episode_rewards)/len(episode_rewards), max(episode_rewards), min(episode_rewards)))

/usr/local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


KeyboardInterrupt: 

In [ ]:
input.plot_stats(n=10)
hidden.plot_stats(n=10)
output.plot_stats(n=10)